# Model: Desperado v0

**Model Parameters:**
* Ensemble model with simple averaging
* Including predictions from:
    * Foxhoud_v0
    * Deadcell_v0
    * Cobra_v0

---
# Load data & libraries

In [1]:
# TOP VERSION
MODEL_NAME = "desperado"
VERSION = 0

In [2]:
# import dependencies
import pandas as pd
from numerapi import NumerAPI
from utils.api_keys import PUBLIC_ID, SECRET_KEY

In [3]:
# instantiate numerai api
napi = NumerAPI(public_id=PUBLIC_ID, secret_key=SECRET_KEY)
current_round = napi.get_current_round()
print(f"Current round #: {current_round}")

Current round #: 314


In [4]:
# read val predictions from models
foxhound_val = pd.read_csv(f"predictions/dh_foxhound_v0_val_preds_{current_round}.csv")
deadcell_val = pd.read_csv(f"predictions/dh_deadcell_v0_val_preds_{current_round}.csv")
cobra_val = pd.read_csv(f"predictions/dh_cobra_v0_val_preds_{current_round}.csv")

print(f"""
    Foxhound predictions : {len(foxhound_val)}
    Deadcell pedictions  : {len(deadcell_val)}
    Cobra pedictions     : {len(cobra_val)}
    """)


    Foxhound predictions : 2203644
    Deadcell pedictions  : 2203644
    Cobra pedictions     : 2203644
    


In [5]:
# read live predictions from models
foxhound_live = pd.read_csv(f"predictions/dh_foxhound_v0_live_preds_{current_round}.csv")
deadcell_live = pd.read_csv(f"predictions/dh_deadcell_v0_live_preds_{current_round}.csv")
cobra_live = pd.read_csv(f"predictions/dh_cobra_v0_live_preds_{current_round}.csv")

print(f"""
    Foxhound predictions : {len(foxhound_live)}
    Deadcell pedictions  : {len(deadcell_live)}
    Cobra pedictions     : {len(cobra_live)}
    """)


    Foxhound predictions : 5317
    Deadcell pedictions  : 5317
    Cobra pedictions     : 5317
    


---
# Create ensemble prediction

In [6]:
# model name
model_name = f"dh_{MODEL_NAME}_v{VERSION}"
model_name

'dh_desperado_v0'

In [7]:
# average val preds
desperado_val = foxhound_val.merge(
    right=deadcell_val, how='inner', on="id").merge(
    right=cobra_val, how='inner', on="id")

desperado_val.columns = ["id", "foxhound", "deadcell", "cobra"]
desperado_val["prediction"] = (
    desperado_val["foxhound"] + 
    desperado_val["deadcell"] + 
    desperado_val["cobra"]
    ) / 3
desperado_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2203644 entries, 0 to 2203643
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id          object 
 1   foxhound    float64
 2   deadcell    float64
 3   cobra       float64
 4   prediction  float64
dtypes: float64(4), object(1)
memory usage: 100.9+ MB


In [8]:
# average live preds
desperado_live = foxhound_live.merge(
    right=deadcell_live, how='inner', on="id").merge(
    right=cobra_live, how='inner', on="id")

desperado_live.columns = ["id", "foxhound", "deadcell", "cobra"]
desperado_live["prediction"] = (
    desperado_live["foxhound"] + 
    desperado_live["deadcell"] + 
    desperado_live["cobra"]
    ) / 3
desperado_live.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5317 entries, 0 to 5316
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          5317 non-null   object 
 1   foxhound    5317 non-null   float64
 2   deadcell    5317 non-null   float64
 3   cobra       5317 non-null   float64
 4   prediction  5317 non-null   float64
dtypes: float64(4), object(1)
memory usage: 249.2+ KB


In [9]:
# save ensemble val preds
desperado_val = desperado_val[["id", "prediction"]].set_index("id")
desperado_val.to_csv(f"predictions/{model_name}_val_preds_{current_round}.csv")

---
# Save predictions

In [10]:
# save ensemble live preds
desperado_live = desperado_live[["id", "prediction"]].set_index("id")
desperado_live.to_csv(f"predictions/{model_name}_live_preds_{current_round}.csv")